In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [15]:

class Uni_RNN(nn.Module):
    #hidden1: chatpgt 512-384
    #hidden2: chatpgt 256-128
    def __init__(self,embedding_model,hidden1,hidden2):
        super().__init__()
        embedding_dim=768
        self.embedding_model = BertModel.from_pretrained(embedding_model)        
        self.LSTM= nn.LSTM(embedding_dim,hidden1, batch_first=True)
        self.LSTM2= nn.LSTM(hidden1,hidden2, batch_first=True)
        self.fc_pinicial= nn.Linear(hidden2,1)
        self.fc_cap = nn.Linear(hidden2, 1)         # sigmoid
        self.fc_pfinal = nn.Linear(hidden2, 3) 
                                           
    def forward(self,data,inferencia=False):
        data= torch.tensor(data)
        data= self.embedding_model(data)
        data, _=self.LSTM(data)
        data, _=self.LSTM2(data)
        final= data[:, -1, :]

        output_cap= torch.sigmoid(self.fc_cap(final))
        output_punt_inicial= torch.sigmoid(self.fc_pinicial(final))
        output_punt_final=F.SoftMax(self.fc_pfinal(final))

        if inferencia:
            output_punt_final=torch.argmax(punt_final_logits,dim=-1)

        return output_cap,output_punt_inicial,output_punt_final


In [7]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class dataset_subtitulos(Dataset):
    def __init__(self, path):
        data = pd.read_parquet(
            path,
            columns=[
                "token_id",
                "punt_inicial",
                "puntos",
                "comas",
                "cierre_pregunta",
                "capitalizacion",
            ],
        )

        self.X = torch.tensor(data["token_id"].values, dtype=torch.long)
        self.y = torch.tensor(
            data[
                ["punt_inicial", "puntos", "comas", "cierre_pregunta", "capitalizacion"]
            ].values,
            dtype=torch.long,
        )

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):  
        return self.X[idx], self.y[idx]


sub_data = dataset_subtitulos("datos_modelo.parquet")



#data_loader

In [8]:
data_loader_train=DataLoader(sub_data,batch_size=128,shuffle=True)

In [17]:
#TRAINING LOOP
model=Uni_RNN("bert-base-multilingual-cased",512,256)

criterion = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=0.001)

  